In [1]:
import MetaTrader5 as mt5
import mt5utilities as util
import config as cfg
from db_manager import DatabaseManager
from datetime import datetime, timedelta
import time
import logging
import traceback

In [2]:

class Bot:
    def __init__(self, mt5_connector, market_status, symbol, timeframe, from_data, to_data, lot, deviation, magic1, magic2, magic3, tp_pips, atr_sl_multiplier, atr_period, max_dist_atr_multiplier, trail_atr_multiplier, webhook_url):
        self.mt5_connector = mt5_connector
        self.market_status = market_status
        self.symbol = symbol
        self.timeframe = timeframe
        self.from_data = from_data
        self.to_data = to_data
        self.lot = lot
        self.deviation = deviation
        self.magic1 = magic1
        self.magic2 = magic2
        self.magic3 = magic3
        self.tp_pips = tp_pips
        self.atr_sl_multiplier = atr_sl_multiplier
        self.atr_period = atr_period
        self.max_dist_atr_multiplier = max_dist_atr_multiplier
        self.trail_atr_multiplier = trail_atr_multiplier
        self.webhook_url = webhook_url
        self.should_stop = False
        self.positions = {}
        self.retracement_trade_executed = False

        self.level_broken = False


        self.username = 'Tracy'
        self.box = None
        self.daily_trade_info = None


        self.retracment_notice = False


        #initialize data fetcher
        self.data_fetcher = util.DataFetcher(mt5_connector, symbol, timeframe, from_data, to_data)

        #initialize position manager
        self.position_manager = util.OpenPositionManager(mt5_connector, self.symbol, self.timeframe, self.from_data, self.to_data, self.atr_period, self.max_dist_atr_multiplier, self.atr_sl_multiplier, self.trail_atr_multiplier)

        #initialize Messanger
        self.messanger = util.Messenger(self.webhook_url, self.username)

        self.trade_history = util.TradeHistory(mt5_connector, self.symbol)

        #initialize flags
        self.data_fetched = False
        self.box_calculated = False
        self.levels_calculated = False
        self.trade_executed = False
        self.trading_notification = False
        self.trade_signal_notification = False
        self.position_manager_nofitication = False

        self.daily_data_reset = False

        self.db_manager = DatabaseManager('trades.db')

        self.positions_loaded = False

        # Configure the logger
        self.logger = logging.getLogger('Bot')

        # File handler
        file_handler = logging.FileHandler('bot.log')
        file_handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)s:%(message)s'))

        # Stream handler
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)s:%(message)s'))

        # Add handlers to the logger
        if not self.logger.handlers:  # Prevent logging from duplicating messages
            self.logger.addHandler(file_handler)
            self.logger.addHandler(stream_handler)

        self.logger.setLevel(logging.INFO)


        error_handler = logging.FileHandler('error.log')
        error_handler.setLevel(logging.ERROR)
        error_handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)s: %(message)s'))
        self.logger.addHandler(error_handler)


        # Define the schema for your opened_trades table, now with added fields
        trade_schema = """
            date_time TEXT,
            ticket_id INTEGER,
            symbol TEXT,
            trade_type TEXT,
            open_price REAL,
            magic_number INTEGER,
            date_time_close TEXT,
            close_price REAL,
            profit_loss REAL,
            status BOOLEAN
        """
        # Create the table in the database
        self.db_manager.create_table('opened_trades', trade_schema)

        # Define the schema for your closed_trades table, now with added fields
        closed_trade_schema = """
            date_time_open TEXT,
            ticket_id INTEGER,
            trade_type TEXT,
            open_price REAL,
            magic_number INTEGER,
            date_time_close TEXT,
            close_price REAL,
            profit_loss REAL,
            status BOOLEAN
        """
        # Create the table in the database
        self.db_manager.create_table('closed_trades', closed_trade_schema)


   
                    
                    # Update the current price in the Position object
                    position.current_price = open_position_data.iloc[0, 13]

                    # Perform some actions based on its magic number
                    if not position.magic_number:
                    
                        update_stop = self.position_manager.calculate_manual_stop(position)
                        if update_stop:
                            self.logger.info("-------------------------------------------------")
                            self.logger.info(f'{self.symbol}: stop_loss set : {update_stop.comment}')

                        # trail position
                        if position.take_profit != 0.0:
                            update_result = self.position_manager.calculate_atr_trailing_stop(position)
                        
                            if update_result:
                                self.logger.info("-------------------------------------------------")
                                self.logger.info(f'{self.symbol}: manual Position Trailled : {update_result.comment}')

                    elif position.magic_number == self.magic2:
                        # trail position with magic number
                        update_result = self.trail_stop(position)
                        if update_result:
                            self.logger.info("-------------------------------------------------")
                            self.logger.info(f'{self.symbol}: Position Trailled : {update_result.comment}')

                else:
                 # The position is closed

                    # Fetch the closed trade from the history using ticket ID
                    history_orders = self.trade_history.get_history(ticket=ticket_id)

                    if history_orders is not None and len(history_orders) > 0:
                        # Assuming you want the first order if there are multiple
                        closed_trade = history_orders[0]

                        # Update the position with the close details
                        position.close_time = closed_trade.time_done  # Assuming 'time_done' is the attribute for close time
                        position.close_price = closed_trade.price_current  # Assuming 'price_current' is the attribute for close price

                        if position.trade_type == 'Buy':
                            position.profit_loss = (position.close_price - position.open_price) * position.lot
                        elif position.trade_type == 'Sell':
                            position.profit_loss = (position.open_price - position.close_price) * position.lot

                        position.status = True

                        # Update the position in the database
                        self.update_position_db(position)

                        # Remove the position from the positions list
                        del self.positions[ticket_id]
                    else:
                        # Handle the case when no history orders are found for the given ticket ID
                        self.logger.error("No history orders found for ticket ID {}".format(ticket_id))

            except Exception as e:
                # Handle the exception (print or log the error)
                self.logger.error('An error occurred: %s', e)
                tb = traceback.format_exc()  # Get the traceback
                self.logger.error('An error occurred: %s\n%s', e, tb)  # Log the error and traceback



    def reset_data(self):
           
        # Reset the trading data
        self.box = None
        self.daily_trade_info = None


        # Reset flags
        self.data_fetched = False
        self.box_calculated = False
        self.levels_calculated = False
        self.trade_executed = False
        self.trading_notification = False
        self.trade_signal_notification = False
        self.position_manager_notification = False
        self.retracement_trade_executed = False
        self.level_broken = False


        self.daily_data_reset = True
        # Send a reset message
        self.messanger.send(f'{self.symbol}:Today data reset:{self.daily_data_reset}')
        self.logger.info('------------------------------------------------------------') 
        self.logger.info(f'{self.symbol}:Today data reset:{self.daily_data_reset}')
           


-------------------------------
initialize: True
----------------------------
logged in:  True


In [ ]:
util.TradeHistory(connector, cfg.symbols)